In [22]:
from langchain_openai import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

MODEL = "gpt-4o-mini"

- parser

In [3]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

In [5]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [6]:
reply = "red, blue, green"
output_parser.parse(reply)

['red', 'blue', 'green']

- example 

In [17]:
human_template = "{request}\n\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate([human_prompt])

In [26]:
model_request = chat_prompt.format_prompt(request="give me 5 characteristics about dogs",
                                          format_instructions=output_parser.get_format_instructions())

In [27]:
chat = ChatOpenAI(model=MODEL)
result = chat.invoke(model_request)

In [29]:
result.content

'loyalty, intelligence, playfulness, sociability, adaptability'

In [30]:
output_parser.parse(result.content)

['loyalty', 'intelligence', 'playfulness', 'sociability', 'adaptability']

- datetime parser

In [31]:
from langchain.output_parsers import DatetimeOutputParser

In [32]:
output_parser = DatetimeOutputParser()

In [34]:
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0718-05-14T08:53:59.938892Z, 1892-11-16T12:04:09.424294Z, 1308-09-28T03:35:53.313484Z

Return ONLY this string, no other words!


In [37]:
human_template = "{request}\n\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate([human_prompt])

In [40]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                         format_instructions=output_parser.get_format_instructions()))

Human: When was the 13th Amendment ratified in the US?

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1822-09-21T18:40:48.360039Z, 0757-02-16T22:19:36.879169Z, 1379-04-08T03:41:05.669914Z

Return ONLY this string, no other words!


In [44]:
# FUNCIONA COM .format E .format_prompt
request = (chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                         format_instructions=output_parser.get_format_instructions()))

In [52]:
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)
result = chat.invoke(request)

In [54]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

In [58]:
wrong_output = 'The 13th Amendment was ratified in the US on December 6, 1865.\n\nThe datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"'

try:
    output_parser.parse(wrong_output)
except Exception as e:
    print(e)

Could not parse datetime string: The 13th Amendment was ratified in the US on December 6, 1865.

The datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"


In [59]:
from langchain.output_parsers import OutputFixingParser

In [60]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=chat) 

In [64]:
new_parser.parse(wrong_output)

datetime.datetime(1865, 12, 6, 0, 0)

- pydantic

In [66]:
from langchain.output_parsers import PydanticOutputParser

In [67]:
from pydantic import BaseModel, Field

In [78]:
class Scientist(BaseModel):

    name: str = Field(description="Name of a Scientist")
    discoveries: list = Field(description="Python list of discoveries")

class Cientista(BaseModel):

    nome: str = Field(description="Nome de um cientista")
    descobertas: list = Field(description="Lista python com descobertas")

In [88]:
parser = PydanticOutputParser(pydantic_object=Cientista) 

In [89]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"nome": {"description": "Nome de um cientista", "title": "Nome", "type": "string"}, "descobertas": {"description": "Lista python com descobertas", "items": {}, "title": "Descobertas", "type": "array"}}, "required": ["nome", "descobertas"]}
```


In [90]:
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n\n{format_instructions}")

chat_prompt = ChatPromptTemplate([human_prompt])

In [95]:
request = chat_prompt.format_prompt(request="Me fale de um cientista famoso",
                                    format_instructions=parser.get_format_instructions())


In [96]:
result = chat.invoke(request)

In [97]:
parser.parse(result.content)

Cientista(nome='Albert Einstein', descobertas=['Teoria da Relatividade', 'Efeito Fotoelétrico'])

: 